In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error

In [ ]:
torch.manual_seed(42)

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
data=pd.read_csv('/content/bike sharing.csv')
data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [ ]:
data.isna().sum()

,0
instant,0
dteday,0
season,0
yr,0
mnth,0
hr,0
holiday,0
weekday,0
workingday,0
weathersit,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


In [ ]:
data['dteday']=pd.to_datetime(data['dteday'])

data['day'] = data['dteday'].dt.day
data['month'] = data['dteday'].dt.month
data['year'] = data['dteday'].dt.year


X=data.drop(columns=['instant', 'dteday', 'casual', 'registered'],axis=1)
y=data['cnt']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#creating custom dataset class
class bikedataset(Dataset):
  def __init__(self,features,targets):
    self.features=torch.tensor(features.values.astype(np.float32),dtype=torch.float32) # Changed here
    self.targets=torch.tensor(targets.values,dtype=torch.float32)

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self,idx):
    x=self.features[idx]
    y=self.targets[idx]
    return x,y

In [ ]:
#creating train and test dataset object
train_dataset=bikedataset(X_train,y_train)
test_dataset=bikedataset(X_test,y_test)


#creating train and test dataloader
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=False,pin_memory=True)

In [ ]:
len(train_loader)

218

In [ ]:
len(test_loader)

55

In [ ]:
#creating custom nn
class MyNN(nn.Module):
    def __init__(self, num_features):
        super(MyNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
learning_rate=0.001
epochs=50

#instatiate the model
model=MyNN(X_train.shape[1])
model=model.to(device)

criterion=nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [ ]:
#training loop
for epoch in range(epochs):
  total_epoch_loss=0
  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    outputs=model(batch_features)
    loss=criterion(outputs,batch_labels)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    total_epoch_loss+=loss.item()
  print(f'Epoch {epoch+1}/{epochs}, Loss: {total_epoch_loss/len(train_loader)}')


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50, Loss: 35601.16576476491
Epoch 2/50, Loss: 34080.60679472477
Epoch 3/50, Loss: 33841.58214772076
Epoch 4/50, Loss: 33958.43572158113
Epoch 5/50, Loss: 34085.14338446101
Epoch 6/50, Loss: 33917.61607654816
Epoch 7/50, Loss: 33869.76251612672
Epoch 8/50, Loss: 33899.56782181766
Epoch 9/50, Loss: 33986.014433414566
Epoch 10/50, Loss: 33830.14570491686
Epoch 11/50, Loss: 34036.04491291571
Epoch 12/50, Loss: 33991.90227207569
Epoch 13/50, Loss: 33997.573654314794
Epoch 14/50, Loss: 34017.08023491256
Epoch 15/50, Loss: 33746.50532181766
Epoch 16/50, Loss: 33887.376119911125
Epoch 17/50, Loss: 33862.737851204125
Epoch 18/50, Loss: 33785.56365126863
Epoch 19/50, Loss: 33885.49544420155
Epoch 20/50, Loss: 33873.35947355218
Epoch 21/50, Loss: 33750.46433307053
Epoch 22/50, Loss: 33848.33591062213
Epoch 23/50, Loss: 33893.398782432625
Epoch 24/50, Loss: 33787.82108299885
Epoch 25/50, Loss: 33888.182276734515
Epoch 26/50, Loss: 33787.04283436066
Epoch 27/50, Loss: 33864.40434167145
Epoc

In [ ]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=16, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [ ]:
predictions=[]
targets=[]
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    outputs=model(batch_features)

    predictions.extend(outputs.cpu().numpy())
    targets.extend(batch_labels.cpu().numpy())

predictions=np.array(predictions)
targets=np.array(targets)

r2 = r2_score(targets, predictions)
mse = mean_squared_error(targets, predictions)

# Display results
print(f" R² Score       : {r2:.4f}")
print(f" MSE (Loss)     : {mse:.4f}")


 R² Score       : 0.0214
 MSE (Loss)     : 30989.2344


In [ ]:
predictions=[]
targets=[]
with torch.no_grad():
  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    outputs=model(batch_features)

    predictions.extend(outputs.cpu().numpy())
    targets.extend(batch_labels.cpu().numpy())

predictions=np.array(predictions)
targets=np.array(targets)

r2 = r2_score(targets, predictions)
mse = mean_squared_error(targets, predictions)

# Display results
print(f" R² Score       : {r2:.4f}")
print(f" MSE (Loss)     : {mse:.4f}")

 R² Score       : 0.0180
 MSE (Loss)     : 32605.6426
